<a href="https://colab.research.google.com/github/vilcagamarracf/Inv_arroz_METRIC/blob/main/Inv_Arroz_METRIC_Pruebas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pruebas

#### Inicio

##### Credenciales

In [1]:
# # Ingresa tus credenciales de Google Earth Engine para comenzar
# import ee
# ee.Authenticate()

# # Inicio
# ee.Initialize()

# # Mensaje de login
# print('\nYa estás logeado.')
# print('GEE Version:', ee.__version__)

In [2]:
#@markdown *Credenciales Google Earth Engine: Forma rápida*
import os 
credential = '{"refresh_token":"1//09lhqedCNpNDsCgYIARAAGAkSNwF-L9IraEP8FyJma46WSiEHhOMsZqSoTPJsi3Lar0C4nZUwj2QCZ_i66-McVzy4NmMalQL17KM"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)
import ee
ee.Initialize()
print('Ya estás logeado.')
print('GEE Version:',ee.__version__)

Ya estás logeado.
GEE Version: 0.1.290


##### geemap

In [3]:
# Instalar geemap
%%capture
!pip install -U geemap

Documentación `geemap`
- [geemap - key features](https://geemap.org/notebooks/00_geemap_key_features/)

In [46]:
# Interfaz interactiva con geemap 
import geemap
# import geemap.eefolium as geemap
Map = geemap.Map(basemap='HYBRID', center=(-9.125, -74.396), zoom=5)
# Map = geemap.Map(basemap='OpenStreetMap.Mapnik', center=(-9.125, -74.396), zoom=5) # Para visualización minimalista
# Map

##### Funciones

In [5]:
# Librerías
import pandas as pd
from ipywidgets import interact, fixed

In [6]:
# Mejorar visibilidad de tablas en Colab
%load_ext google.colab.data_table

## Visualizaciones

### Distrito de Chongoyape

In [101]:
%%capture
!pip install geopandas

In [102]:
import geopandas as gpd

# leer archivo con geopandas.read_file()
distritosPeru = gpd.read_file('drive/MyDrive/Archivos_SIG/Limite_Distrital/Distritos_Peru_WGS84.shp')
# distritosPeru

# Filtrar el distrito
lim_chongoyape = distritosPeru[distritosPeru.NOMBDIST == 'CHONGOYAPE']

# Crear un featureCollection mediante GeoJSON: geopandas.read_file().__geo_interface__
ee_chongoyape = ee.FeatureCollection(lim_chongoyape.__geo_interface__)
type(ee_chongoyape) # ee.featurecollection.FeatureCollection

ee.featurecollection.FeatureCollection

In [103]:
# @markdown Visualización con `geemap`: Departamento de Lambayeque

# 24 Departamentos del Perú
shpPeru = ee.FeatureCollection('users/CesarVilca/Departamentos_Peru')

# Departamento de Lambayeque
shpDep = shpPeru.filter(ee.Filter.eq('NOMBDEP', 'LAMBAYEQUE'))

# Geometrías
roiPeru = shpPeru.geometry()
roiDep  = shpDep.geometry()

# https://htmlcolors.com/ para modificar el color
roiPeruDraw = shpPeru.draw(
    color = "D8EED0", strokeWidth = 1, pointRadius = 2) 
roiDepDraw  = shpDep.draw(
    color = "00FF00", strokeWidth = 2, pointRadius = 2) 
roiChongoyapeDraw  = ee_chongoyape.draw(
    color = "00FFFF", strokeWidth = 2, pointRadius = 2) 

# Visualización: geemap
Map = geemap.Map(basemap='HYBRID', layer_ctrl=True) # OpenStreetMap.Mapnik
Map.centerObject(ee_chongoyape, 9) # Map.setCenter(lon, lat, zoom)
Map.addLayer(roiPeruDraw, {}, name = 'Perú')
Map.addLayer(roiDepDraw, {}, name = 'Lambayeque')
Map.addLayer(roiChongoyapeDraw, {}, name = 'Chongoyape')

Map

Map(center=[-6.626857372606788, -79.4645728499761], controls=(WidgetControl(options=['position', 'transparent_…

In [7]:
chongoyape = ee.Geometry.Point([-79.389, -6.639])

### Landsat 8

In [ ]:
# @markdown Función: `generar_reporte_landsat8_T1_SR(path, row, year)`
def generar_reporte_landsat8_T1_SR(path, row, year):

  ID_snippet_name = "LANDSAT/LC08/C01/T1_SR"
  ## Filtrado de ee.imageCollection
  icol_sr = (
      ee.ImageCollection(ID_snippet_name)\
      .filterDate(str(year), str(year+1))\
      .filterMetadata('WRS_PATH', 'equals', path)\
      .filterMetadata('WRS_ROW', 'equals', row)
  )

  ## Generando campos para la tabla
  # Lista con fechas (en milisegundos)
  lista_fechas = icol_sr.aggregate_array('system:time_start').getInfo()

  # Lista con ID's
  imgCol_ids = [
                ID_snippet_name + 
                '/'+ 
                i for i in icol_sr.aggregate_array("system:index").getInfo()]

  # Tabla con pandas
  # import pandas as pd

  df = pd.DataFrame(lista_fechas, columns = ['millis'])

  df["Landsat ID"] = imgCol_ids
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Fecha Precisa'] = pd.DatetimeIndex(df['Timestamp']) # Con hora
  df['Fecha Corta'] = pd.DatetimeIndex(df['Timestamp']).date
  df['Año'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Mes'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Día'] = pd.DatetimeIndex(df['Timestamp']).day
  df['Hora'] = pd.DatetimeIndex(df['Timestamp']).hour
  df['Día Juliano'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  # df['Sensor'] = 'OLI/TIRS'
  
  df["Zenith Angle"] = icol_sr.aggregate_array('SOLAR_ZENITH_ANGLE').getInfo()
  df["Porcentaje Nubes (%)"] = icol_sr.aggregate_array('CLOUD_COVER').getInfo()
  
  df = df.drop(columns=['millis', 'Timestamp'])
  # df.to_csv('datos_2020_L8_SR.csv')
  
  return df

In [ ]:
# @markdown Función `visualizar_mes_L8_SR(mes)`

def visualizar_mes_L8_SR(mes, df):
  
  # Del dataframe obtenemos las imagenes de acuerdo al mes que queramos
  lista_imagenes = df[df['Mes'] == mes]['Landsat ID'].tolist()

  # Parametros de visualización RGB
  vis_rgb = {'min': 0.0, 'max': 0.3, 'gamma':1.4, 'bands': ['B4', 'B3', 'B2']}
  
  # geemap para la visualización
  Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)

  Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)
  for i in lista_imagenes:
    Map.addLayer(ee.Image(i).multiply(0.0001), vis_rgb, f'Imagen {i[-2:]}') # .clip(roiDep)

  # Visualizar al último el distrito de Chongoyape
  Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw
  
  return Map

In [ ]:
df1 = generar_reporte_landsat8_T1_SR(path=10, row=65, year=2021)
mes = df1['Mes'].unique().tolist()

interact(visualizar_mes_L8_SR, mes=mes, df=fixed(df1));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=1), Output())…

### Sentinel-2

In [8]:
# @markdown Función: `generar_reporte_S2_SR(roi, year)`
def generar_reporte_S2_SR(roi, year):

  ID_snippet_name = 'COPERNICUS/S2_SR'

  ## Filtrado de ee.imageCollection
  icol_sr = (
      ee.ImageCollection(ID_snippet_name)\
      .filterDate(str(year), str(year+1))\
      .filterBounds(roi)
  )

  ## Generando campos para la tabla
  # Lista con fechas (en milisegundos)
  lista_fechas = icol_sr.aggregate_array('system:time_start').getInfo()

  # Lista con ID's
  imgCol_ids = [f'{ID_snippet_name}/{i}' for i in icol_sr.aggregate_array("system:index").getInfo()]

  # Tabla con pandas
  # import pandas as pd

  df = pd.DataFrame(lista_fechas, columns = ['millis'])

  df['Sentinel-2 ID'] = imgCol_ids
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Fecha Precisa'] = pd.DatetimeIndex(df['Timestamp']) # Con hora
  df['Fecha Corta'] = pd.DatetimeIndex(df['Timestamp']).date
  df['Año'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Mes'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Día'] = pd.DatetimeIndex(df['Timestamp']).day
  df['Hora'] = pd.DatetimeIndex(df['Timestamp']).hour
  df['Día Juliano'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  # df['Sensor'] = 'OLI/TIRS'
  
  df["Zenith Angle"] = icol_sr.aggregate_array('MEAN_SOLAR_ZENITH_ANGLE').getInfo()
  df["Porcentaje Nubes (%)"] = icol_sr.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()
  
  df = df.drop(columns=['millis', 'Timestamp'])
  # df.to_csv('datos_2020_L8_SR.csv')

  return df # ,icol_sr

In [116]:
# @markdown Función `visualizar_mes_S2_SR(mes)`

def visualizar_mes_S2_SR(mes, df):
  
  # Del dataframe obtenemos las imagenes de acuerdo al mes que queramos
  lista_imagenes = df[df['Mes'] == mes]['Sentinel-2 ID'].tolist()

  # Parametros de visualización RGB
  vis_rgb = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']} # 'gamma':1.4
  
  # geemap para la visualización 
  Map = geemap.Map(layer_ctrl=True) # basemap='HYBRID',OpenStreetMap.Mapnik
  Map.centerObject(ee_chongoyape, 12)  #11, Map.setCenter(-79.809, -6.746, 9)

  # Agregar imágenes por mes
  for i in lista_imagenes:
    Map.addLayer(ee.Image(i).multiply(0.0001).clip(ee_chongoyape), vis_rgb, f'Imagen {i[21:23]}/{i[23:25]}') # {i} .clip(roiDep)
 
  # Visualizar al último el distrito de Chongoyape
  Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw
  
  return Map

In [118]:
df1 = generar_reporte_S2_SR(chongoyape, year=2021)
mes = df1['Mes'].unique().tolist()

interact(visualizar_mes_S2_SR, mes=mes, df=fixed(df1));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=1), Outpu…

### Indices de Vegetación

In [72]:
# @markdown Paletas de colores para NDVI y EVI
#------------- Paletas de NDVI ---------------------------------------
# @markdown - `BWG` : Blue, White, Green para valores -1 a 1
BWG = ['blue', 'white', 'green']

# @markdown - `ndviPalette02` : GIMMS NDVI From AVHRR Sensors (3rd Generation)
# https://developers.google.com/earth-engine/datasets/catalog/NASA_GIMMS_3GV0?hl=en
GIMMS_NDVI = ['000000', 'f5f5f5', '119701']

# @markdown - `ndviPalette03` : MOD13A1.006 Terra Vegetation Indices 16-Day Global 500m
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A1?hl=en#description
ndviPalette03 = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
]

# @markdown - `yuri_ndvi_pal` : Paleta NDVI (Yuri SIG) 20 Colores
yuri_ndvi_pal = [
    'ffffff', 'fffcff', 'fff9ff', 'fff7ff', 'fff4ff', 'fff2ff', 'ffefff',
    'ffecff', 'ffeaff', 'ffe7ff', 'ffe5ff', 'ffe2ff', 'ffe0ff', 'ffddff',
    'ffdaff', 'ffd8ff', 'ffd5ff', 'ffd3ff', 'ffd0ff', 'ffceff', 'ffcbff',
    'ffc8ff', 'ffc6ff', 'ffc3ff', 'ffc1ff', 'ffbeff', 'ffbcff', 'ffb9ff',
    'ffb6ff', 'ffb4ff', 'ffb1ff', 'ffafff', 'ffacff', 'ffaaff', 'ffa7ff',
    'ffa4ff', 'ffa2ff', 'ff9fff', 'ff9dff', 'ff9aff', 'ff97ff', 'ff95ff',
    'ff92ff', 'ff90ff', 'ff8dff', 'ff8bff', 'ff88ff', 'ff85ff', 'ff83ff',
    'ff80ff', 'ff7eff', 'ff7bff', 'ff79ff', 'ff76ff', 'ff73ff', 'ff71ff',
    'ff6eff', 'ff6cff', 'ff69ff', 'ff67ff', 'ff64ff', 'ff61ff', 'ff5fff',
    'ff5cff', 'ff5aff', 'ff57ff', 'ff55ff', 'ff52ff', 'ff4fff', 'ff4dff',
    'ff4aff', 'ff48ff', 'ff45ff', 'ff42ff', 'ff40ff', 'ff3dff', 'ff3bff',
    'ff38ff', 'ff36ff', 'ff33ff', 'ff30ff', 'ff2eff', 'ff2bff', 'ff29ff',
    'ff26ff', 'ff24ff', 'ff21ff', 'ff1eff', 'ff1cff', 'ff19ff', 'ff17ff',
    'ff14ff', 'ff12ff', 'ff0fff', 'ff0cff', 'ff0aff', 'ff07ff', 'ff05ff',
    'ff02ff', 'ff00ff', 'ff00ff', 'ff0af4', 'ff15e9', 'ff1fdf', 'ff2ad4',
    'ff35c9', 'ff3fbf', 'ff4ab4', 'ff55aa', 'ff5f9f', 'ff6a94', 'ff748a',
    'ff7f7f', 'ff8a74', 'ff946a', 'ff9f5f', 'ffaa55', 'ffb44a', 'ffbf3f',
    'ffc935', 'ffd42a', 'ffdf1f', 'ffe915', 'fff40a', 'ffff00', 'ffff00',
    'fffb00', 'fff700', 'fff300', 'fff000', 'ffec00', 'ffe800', 'ffe400',
    'ffe100', 'ffdd00', 'ffd900', 'ffd500', 'ffd200', 'ffce00', 'ffca00',
    'ffc600', 'ffc300', 'ffbf00', 'ffbb00', 'ffb700', 'ffb400', 'ffb000',
    'ffac00', 'ffa800', 'ffa500', 'ffa500', 'f7a400', 'f0a300', 'e8a200',
    'e1a200', 'd9a100', 'd2a000', 'ca9f00', 'c39f00', 'bb9e00', 'b49d00',
    'ac9c00', 'a59c00', '9d9b00', '969a00', '8e9900', '879900', '7f9800',
    '789700', '709700', '699600', '619500', '5a9400', '529400', '4b9300',
    '439200', '349100', '2d9000', '258f00', '1e8e00', '168e00', '0f8d00',
    '078c00', '008c00', '008c00', '008700', '008300', '007f00', '007a00',
    '007600', '007200', '006e00', '006900', '006500', '006100', '005c00',
    '005800', '005400', '005000', '004c00'
]


#------------- Paletas de EVI ---------------------------------------
evi_palette = [
    'ffffff', 'fcd163', '99b718', '66a000', '3e8601', '207401', '056201',
    '004c00', '011301'
]

ndvi_palette = [
    "#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00",
    "#429001", "#006400", "#003B00", "#000000"
]

Más paletas de colores:
- [OpenWeather in Medium: Visualisation of the NDVI index on satellite maps. Custom palettes for agricultural applications](https://openweathermap.medium.com/visualisation-of-the-ndvi-index-on-satellite-maps-custom-palettes-for-agricultural-applications-f99b0652f991)

#### NDVI

In [15]:
# @markdown Función NDVI: `ndvi_icol(image)`

def ndvi_icol(image):
  ndvi = image.expression(
      '(nir - red) / (nir + red)',
      {'red' : image.select('B4').divide(10000),
       'nir' : image.select('B8').divide(10000)}).rename('NDVI')
  return ndvi

# def ndvi_icol(image):
#   ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi')
#   return image.addBands([ndvi])

Escala de NDVI y EVI: -0.2 a 1

In [119]:
# @markdown Ver NDVI Mensual: `ver_ndvi_mensual(df, mes, img)`
def ver_ndvi_mensual(df, mes, img):
  lista_imgs = df['Sentinel-2 ID'][df['Mes'] == mes].tolist()
  print(f'{len(lista_imgs)} imágenes en el mes')

  imagen_str = lista_imgs[img-1]
  imagen_ee = ee.Image(imagen_str)
  imagen_ndvi = ndvi_icol(imagen_ee)
  imagen_rgb = imagen_ee.multiply(0.0001)

  print(f'Fecha: {mes}/{imagen_str[23:25]}')
  # Visualización
  vis_rgb = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']} 
  viz_params = {'min':-0.3, 'max':0.3, 'palette': yuri_ndvi_pal} #paleta}

  left_layer = geemap.ee_tile_layer(imagen_rgb, vis_rgb, 'RGB')
  right_layer = geemap.ee_tile_layer(imagen_ndvi, viz_params, 'NDVI')

  Map = geemap.Map(layer_ctrl=True) # basemap='OpenStreetMap.Mapnik'
  Map.split_map(left_layer, right_layer)
  Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)
  Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw

  # Barra de color
  vmin = viz_params['min']
  vmax = viz_params['max']
  colors = viz_params['palette']
  Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="NDVI")
  # Map.add_legend(builtin_legend='NLCD')

  return Map

Para usar la herramienta Inspector es necesario solo tener una imagen visualizada. 

In [120]:
df1 = generar_reporte_S2_SR(chongoyape, year=2021)
mes = df1['Mes'].unique().tolist()

interact(ver_ndvi_mensual, df=fixed(df1), mes=mes, img=(1,7,1));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=1), IntSl…

##### Prueba

In [42]:
# @markdown Función `visualizar_mes_S2_SR(mes)`

def visualizar_mes_S2_SR(mes, df):
  
  # Del dataframe obtenemos las imagenes de acuerdo al mes que queramos
  lista_imagenes = df[df['Mes'] == mes]['Sentinel-2 ID'].tolist()
  
  # Parametros de visualización RGB
  vis_rgb = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']} # 'gamma':1.4
  ndvi_palette = [
    "#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001",
    "#006400", "#003B00", "#000000"]
  viz_params = {'min':0, 'max':0.5, 'palette': ndvi_palette}

  # geemap para la visualización
  Map = geemap.Map(layer_ctrl=True) # basemap='OpenStreetMap.Mapnik', 
  Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)

  # Agregar imágenes por mes
  for i in lista_imagenes:
    Map.addLayer(ndvi_icol(ee.Image(i)), 
                 viz_params, f'NDVI {i[21:23]}/{i[23:25]}') # {i} .clip(roiDep)
    Map.addLayer(ee.Image(i).multiply(0.0001), 
                 vis_rgb, f'RGB {i[21:23]}/{i[23:25]}') # {i} .clip(roiDep)
 
  # Visualizar al último el distrito de Chongoyape
  Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw
  
  return Map

In [43]:
df1 = generar_reporte_S2_SR(chongoyape, year=2021)
mes = df1['Mes'].unique().tolist()

interact(visualizar_mes_S2_SR, mes=mes, df=fixed(df1));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=1), Outpu…

#### EVI

In [66]:
# @markdown Función EVI: `evi_icol(img)`

def evi_icol(img):
  """
  https://developers.google.com/earth-engine/guides/image_math#expressions
  """
  evi = img.expression('2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
                       {'red' : img.select('B4').divide(10000),
                        'nir' : img.select('B8').divide(10000),
                        'blue': img.select('B2').divide(10000)}).rename('EVI')

  return evi

# def evi_s2(image):
#   nir  = image.select(['B8']).divide(10000)
#   red  = image.select(['B4']).divide(10000)
#   blue = image.select(['B2']).divide(10000)
#   return nir.subtract(red).divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)).multiply(2.5).rename('EVI')

# https://colab.research.google.com/drive/1GCAoapxKzGbT2HcKfKOIf_WDObumoOqJ#scrollTo=5HVOHGM17_8L

In [63]:
# @markdown Ver EVI Mensual: `ver_evi_mensual(df, mes, img)`
def ver_evi_mensual(df, mes, img):
  lista_imgs = df['Sentinel-2 ID'][df['Mes'] == mes].tolist()
  print(f'{len(lista_imgs)} imágenes en el mes')

  imagen_str = lista_imgs[img-1]
  imagen_ee = ee.Image(imagen_str)
  imagen_evi = evi_icol(imagen_ee)
  imagen_rgb = imagen_ee.multiply(0.0001)

  print(f'Fecha: {mes}/{imagen_str[23:25]}')
  # Visualización
  vis_rgb = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']} 
  viz_params = {'min':0.0, 'max':0.5, 'palette': yuri_ndvi_pal} #paleta}

  left_layer = geemap.ee_tile_layer(imagen_rgb, vis_rgb, 'RGB')
  right_layer = geemap.ee_tile_layer(imagen_evi, viz_params, 'EVI')

  Map = geemap.Map(layer_ctrl=True) # basemap='OpenStreetMap.Mapnik'
  Map.split_map(left_layer, right_layer)
  Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)
  Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw
  # Map.add_legend(builtin_legend='NLCD')

  return Map

In [67]:
# geemap: split panel map
df1 = generar_reporte_S2_SR(chongoyape, year=2021)
mes = df1['Mes'].unique().tolist()

interact(ver_evi_mensual, df=fixed(df1), mes=mes, img=(1,7,1));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=1), IntSl…

In [ ]:
def ver_evi_mensual(mes, img):
  lista_imgs = df1['Sentinel-2 ID'][df1['Mes'] == mes].tolist()
  print(f'Hay {len(lista_imgs)} imágenes en este mes')
  evi = evi_icol(ee.Image(lista_imgs[img-1]))

  # viz_params = {'min':0, 'max':0.5, 'palette': yuri_ndvi_pal}
  ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
  viz_params = {'min':0.0, 'max':0.5, 'palette': yuri_ndvi_pal}#ndvi_palette}

  Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
  Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)
  Map.addLayer(evi, viz_params, f'EVI {lista_imgs[img-1][21:23]}/{lista_imgs[img-1][23:25]}')
  Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw

  return Map

interact(ver_evi_mensual, mes=np.arange(1,12,1), img=(1,7,1));

interactive(children=(Dropdown(description='mes', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=1), IntSl…

In [ ]:
evi_mean = evi_icol(ee.Image('COPERNICUS/S2_SR/20211028T153619_20211028T153620_T17MPN'))

viz_params = {'min':-1, 'max':1, 'palette': yuri_ndvi_pal}
# ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
# viz_params = {'min':0, 'max':0.5, 'palette': ndvi_palette}

Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)
Map.addLayer(evi_mean, viz_params, 'EVI')
Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw
Map

Map(center=[-6.639, -79.389], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [ ]:
evi_mean = icol_str.map(evi_icol).mean()

viz_params = {'min':0, 'max':0.5, 'palette': yuri_ndvi_pal}
# ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
# viz_params = {'min':0, 'max':0.5, 'palette': ndvi_palette}

Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)
Map.addLayer(evi_mean, viz_params, 'EVI')
Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw
Map

Map(center=[-6.639, -79.389], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [ ]:
# ndvi_mean = icol_str.map(ndvi_icol).mean()
ndvi_mean = icol_str.map(ndvi_icol).first()

ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
viz_params = {'min':0, 'max':0.5, 'palette': ndvi_palette}

Map = geemap.Map(basemap='OpenStreetMap.Mapnik', layer_ctrl=True)
Map.centerObject(chongoyape, 12)  # Map.setCenter(-79.809, -6.746, 9)
Map.addLayer(ndvi_mean, viz_params, 'NDVI')
Map.addLayer(chongoyape, {'color':'00FF00'}, 'Chongoyape') # roiChongoyapeDraw
Map

Map(center=[-6.639, -79.389], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

## Balance de Energía Superficial - Modelo METRIC

- METRIC (2007) se realizó con imágenes Landsat 7 ETM+ y Landsat 8 empezó a operar en 2013.

### Ecuación del Balance de energía

\

$$
LE = R_n - G - H
$$

Dónde: 
- $LE$ : Energía latente consumida por ET $[W/m²]$
- $R_n$ : Radiación neta (suma de todas las radiaciones de onda corta y larga, sean entrantes o salientes a la superficie) $[W/m²]$
- $G$ : Flujo de calor sensible conducido al suelo $[W/m²]$
- $H$ : Flujo de calor sensible convectado al aire $[W/m²]$

\

### Radiación Neta / Net Radiation $R_n$

Morse et al. 2000:

$$
R_n = (1- \alpha)R_{S\downarrow} + (R_{L\downarrow} - R_{L\uparrow}) - (1- \epsilon_0)R_{L\downarrow}
$$

Dónde: 
- $R_n$ : Flujo de radiación neta $[W/m^2]$
- $\alpha$ : Albedo de superficie
- $R_{S\downarrow}$ : Radiación de onda corta entrante $[W/m^2]$
- $R_{L\downarrow}$ : Radiación de onda larga entrante $[W/m^2]$
- $R_{L\uparrow}$ : Radiación de onda larga saliente $[W/m^2]$
- $\epsilon_0$ : Emisividad del ancho de banda en la superficie / broad-band surface thermal emissivity

The $(1- \epsilon_0)R_{L\downarrow}$ term represents the fraction of incoming long-wave
radiation reflected from the surface.

#### Albedo / Surface Albedo $\alpha_{s}$

#### Radiación de onda corta entrante / Incoming Solar Radiation $R_{S\downarrow}$ 

#### Radiación de onda larga saliente / Outgoing Long-Wave Radiation $R_{L\uparrow}$ 

#### Radiación de onda larga entrante / Incoming Long-Wave Radiation $R_{L\downarrow}$

### Flujo de calor del suelo / Soil Heat Flux $G$

### Flujo de calor sensible / Sensible Heat Flux $H$

### $\lambda{ET}$ y $ET$ instantánea

### Evapotranspiración de cultivo $ET_c$ por día